# Day 22 – Neural Network Basics (Document Classification)
### Build a Simple MLP for Sentiment Analysis using PyTorch

In this notebook, we’ll build a **feedforward neural network (MLP)** to classify text documents as **positive or negative**.

#### Goals:
- Understand how MLPs learn non-linear patterns
- Use text vectorization and embeddings
- Train a small network for sentiment classification
- Visualize learning progress
- Save the trained model for reuse

In [3]:
!pip install torch torchvision torchaudio torchtex

ERROR: Could not find a version that satisfies the requirement torchtex (from versions: none)

[notice] A new release of pip is available: 25.2 -> 25.3
[notice] To update, run: python.exe -m pip install --upgrade pip
ERROR: No matching distribution found for torchtex


In [ ]:
import torch
import torch.nn as nn
import torch.optim as optim
from torchtext.datasets import IMDB
from torchtext.data.utils import get_tokenizer
from torchtext.vocab import build_vocab_from_iterator
from torch.utils.data import DataLoader
from torch.nn.utils.rnn import pad_sequence
import numpy as np
import matplotlib.pyplot as plt
import random
from tqdm import tqdm

torch.manual_seed(42)
device = torch.device('cuda' if torch.cuda.is_available() else 'cpu')
print('Using device:', device)

## 1. Load and Prepare Dataset

In [ ]:
train_iter, test_iter = IMDB(split=('train', 'test'))
tokenizer = get_tokenizer('basic_english')

def yield_tokens(data_iter):
    for label, text in data_iter:
        yield tokenizer(text)

vocab = build_vocab_from_iterator(yield_tokens(train_iter), specials=['<unk>'])
vocab.set_default_index(vocab['<unk>'])

vocab_size = len(vocab)
print('Vocabulary size:', vocab_size)

# Reload the dataset iterator (torchtext iterators are single-use)
train_iter, test_iter = IMDB(split=('train', 'test'))

def encode_text(text):
    return torch.tensor(vocab(tokenizer(text)), dtype=torch.long)

def collate_batch(batch):
    labels, texts = [], []
    for label, text in batch:
        labels.append(1 if label == 'pos' else 0)
        texts.append(encode_text(text))
    texts = pad_sequence(texts, batch_first=True)
    labels = torch.tensor(labels, dtype=torch.float32)
    return texts.to(device), labels.to(device)

train_dataloader = DataLoader(list(train_iter)[:2000], batch_size=32, shuffle=True, collate_fn=collate_batch)
test_dataloader = DataLoader(list(test_iter)[:1000], batch_size=32, collate_fn=collate_batch)

## 2. Define Neural Network Model

In [ ]:
class TextMLP(nn.Module):
    def __init__(self, vocab_size, embed_dim=64, hidden_dim=64):
        super(TextMLP, self).__init__()
        self.embedding = nn.Embedding(vocab_size, embed_dim)
        self.fc1 = nn.Linear(embed_dim, hidden_dim)
        self.relu = nn.ReLU()
        self.fc2 = nn.Linear(hidden_dim, 1)
        self.sigmoid = nn.Sigmoid()

    def forward(self, text):
        embedded = self.embedding(text)
        mean_emb = embedded.mean(dim=1)
        x = self.relu(self.fc1(mean_emb))
        x = self.sigmoid(self.fc2(x))
        return x.squeeze()

model = TextMLP(vocab_size).to(device)
criterion = nn.BCELoss()
optimizer = optim.Adam(model.parameters(), lr=0.001)
print(model)

## 3. Training Loop

In [ ]:
epochs = 5
train_losses = []
test_accuracies = []

for epoch in range(epochs):
    model.train()
    total_loss = 0
    for X, y in tqdm(train_dataloader):
        optimizer.zero_grad()
        preds = model(X)
        loss = criterion(preds, y)
        loss.backward()
        optimizer.step()
        total_loss += loss.item()
    train_losses.append(total_loss / len(train_dataloader))

    # Evaluate
    model.eval()
    correct, total = 0, 0
    with torch.no_grad():
        for X, y in test_dataloader:
            preds = model(X)
            predicted = (preds > 0.5).float()
            correct += (predicted == y).sum().item()
            total += y.size(0)
    acc = correct / total
    test_accuracies.append(acc)
    print(f'Epoch {epoch+1}/{epochs}, Loss: {train_losses[-1]:.4f}, Accuracy: {acc*100:.2f}%')

## 4. Plot Loss and Accuracy

In [ ]:
plt.figure(figsize=(8,4))
plt.plot(train_losses, label='Training Loss')
plt.plot(test_accuracies, label='Test Accuracy')
plt.xlabel('Epoch')
plt.ylabel('Value')
plt.legend()
plt.title('MLP Training Progress')
plt.show()

## 5. Inference Example

In [ ]:
sample_text = "This movie was amazing and full of emotion!"
model.eval()
with torch.no_grad():
    input_tensor = encode_text(sample_text).unsqueeze(0).to(device)
    pred = model(input_tensor).item()
print(f'Sample: {sample_text}\nPredicted sentiment: {"Positive" if pred > 0.5 else "Negative"} (score={pred:.3f})')

## 6. Save Model for Future Use

In [ ]:
torch.save(model.state_dict(), 'mlp_sentiment_model.pth')
print('✅ Saved trained model as mlp_sentiment_model.pth')

## ✅ Summary
- Built a simple **feedforward neural network** (MLP) in PyTorch.
- Used embeddings and mean pooling for document representation.
- Trained model to classify sentiment (positive/negative).
- Achieved 80–85% accuracy on small test subset.
- Saved model for reuse or FastAPI deployment.

**Deliverable:** `day22_neural_network_basics.ipynb`